In [13]:
import ibmmq
import time

In [14]:
QMGR = "QM1"
CHANNEL = "DEV.APP.SVRCONN"
CONN = "localhost(1414)"
QUEUE = "DEV.QUEUE.1"
USER = "app"
PASSWORD = "apppass"

In [15]:
print(f"Connecting to {QMGR} via {CHANNEL}@{CONN} …")
qmgr = ibmmq.connect(QMGR, CHANNEL, CONN, user=USER, password=PASSWORD)
q = ibmmq.Queue(qmgr, QUEUE, 0x10)  # MQOO_OUTPUT

Connecting to QM1 via DEV.APP.SVRCONN@localhost(1414) …


In [16]:
pmo = ibmmq.PMO()
pmo.Persistence = 1  # MQPER_PERSISTENT

for i in range(3):
    md = ibmmq.MD()
    msg = f"Hello from notebook {i+1} @ {time.time():.3f}"
    q.put(msg, md, pmo)
    print("PUT:", msg)

q.close()
qmgr.disconnect()

PUT: Hello from notebook 1 @ 1761927993.969
PUT: Hello from notebook 2 @ 1761927993.972
PUT: Hello from notebook 3 @ 1761927993.973


In [17]:
# connection bits (adjust if needed)
qmgr = ibmmq.connect("QM1","DEV.APP.SVRCONN","localhost(1414)", user="app", password="apppass")
q    = ibmmq.Queue(qmgr, "DEV.QUEUE.1", 0x01)  # MQOO_INPUT_AS_Q_DEF

# GMO: waited get for 5s
gmo = ibmmq.GMO(Options=1, WaitInterval=5000)  # MQGMO_WAIT = 1

# 1 MB buffer is plenty for demo; tune to your payloads
MAX_LEN = 1048576

while True:
    md = ibmmq.MD()
    try:
        m = q.get(MAX_LEN, md, gmo)   # <-- maxLength, MD, GMO (positional!)
        print("GET:", m.decode("utf-8"), " MsgId=", md.MsgId.hex())
    except Exception as e:
        if "2033" in str(e):  # no message within wait
            print("Queue empty / timeout.")
            break
        raise

q.close(); qmgr.disconnect()


GET: Hello from notebook 1 @ 1761927993.969  MsgId= 414d5120514d3120202020202020202082a5046901310240
GET: Hello from notebook 2 @ 1761927993.972  MsgId= 414d5120514d3120202020202020202082a5046902310240
GET: Hello from notebook 3 @ 1761927993.973  MsgId= 414d5120514d3120202020202020202082a5046903310240
Queue empty / timeout.
